In [1]:
%run -i startup.py

In [2]:
%run -i ./functions/dc_transform.py

In [3]:
%run -i ./functions/dc_join.py

data_collections = ['^cursos*','estruturas_curriculares','curriculo_componente','componentes_curriculares','turmas','matriculas_em_componentes']

for dc in data_collections:
    dc_join(db=db,dc=dc)

In [4]:
import re
for dcb in dc_join.dict.keys():
    print('%-35s' % dcb,end='') # %-35s : varchar 35 alinhado à esquerda
    c = []
    for k in dc_join.dict[dcb]['data'].keys():
        if re.findall('^id_.*',k):
            dc_join.dict[dcb]['data'][k] = dc_join.dict[dcb]['data'][k].astype('Int64')
            c.append(k)

    print(c,end='')
    print('')

ufrn_wait/cursos                   ['id_curso', 'id_coordenador', 'id_unidade_responsavel']
ufrn_wait/estruturas_curriculares  ['id_curriculo', 'id_curso']
ufrn_wait/curriculo_componente     ['id_curriculo_componente', 'id_curriculo', 'id_componente_curricular']
ufrn_wait/componentes_curriculares ['id_componente']
ufrn_wait/turmas                   ['id_turma', 'id_componente_curricular', 'id_turma_agrupadora']
ufrn_wait/matriculas_em_componentes['id_turma', 'id_curso']


In [6]:
dc = dc_join.dict

cursos                   = dc['ufrn_wait/cursos']['data']
estruturas_curriculares  = dc['ufrn_wait/estruturas_curriculares']['data']
curriculo_componente     = dc['ufrn_wait/curriculo_componente']['data']
componentes_curriculares = dc['ufrn_wait/componentes_curriculares']['data']
turmas                   = dc['ufrn_wait/turmas']['data']
matriculas_componentes   = dc['ufrn_wait/matriculas_em_componentes']['data']

### Análise de notas por estrutura curricular

In [8]:
getin = lambda df,i,v: df[df[i].isin(v.unique())].reset_index(drop=True)
getre = lambda df,i,v: df[df[i].isin(list(filter(lambda i : re.findall(v,i),df[i])))].reset_index(drop=True)

ect        = getre(cursos,'nome','^CIÊNCIA.*LOGIA')

ect_ec     = getin(estruturas_curriculares, 'id_curso',ect.id_curso)
ect_cc     = getin(curriculo_componente,'id_curriculo',ect_ec.id_curriculo)
ect_ccc    = getin(componentes_curriculares,'id_componente',ect_cc.id_componente_curricular)
ect_turmas = getin(turmas,'id_componente_curricular',ect_ccc.id_componente)
ect_mc     = getin(matriculas_componentes,'id_turma',ect_turmas.id_turma)

In [14]:
ect_turmas_unique = ect_turmas[~ect_turmas.id_turma.duplicated()]

In [15]:
A = ect_turmas_unique[['id_turma','id_componente_curricular','ano','periodo','situacao_turma']]
B = ect_mc
C = pd.merge(A,B,on=['id_turma'])
D = ect_ccc[['id_componente','codigo','modalidade','nome']]
D = D.rename(columns={'id_componente':'id_componente_curricular'})
D = pd.merge(C,D,on=['id_componente_curricular'])
D = D[D['situacao_turma']=='CONSOLIDADA']

In [16]:
nota2float = lambda nota : float(nota.replace(',','.')) if isinstance(nota,str) else nota

In [17]:
D.nota                = D.nota.transform(nota2float)
D.media_final         = D.media_final.transform(nota2float)
D.unidade             = D.unidade.astype('Int64')
D.faltas_unidade      = D.faltas_unidade.astype('Int64')
D.numero_total_faltas = D.numero_total_faltas.astype('Int64')

In [18]:
E = D[['id_turma','id_componente_curricular','descricao','ano','periodo','unidade','nota','faltas_unidade','nome','id_curso','discente']]

In [238]:
n_i = E.id_componente_curricular.nunique()
n_a = E.ano.nunique()
n_p = E.periodo.nunique()
n_u = E.unidade.nunique()

In [158]:
# u_ unique values
# n_ number of values

i_e = {k:v for v,k in enumerate(E.id_componente_curricular.unique())}

c = 0

iapu = np.empty((n_t, n_a, n_p, n_u))

z[:] = np.nan

In [196]:
a = np.empty((3,5),np.float)


a

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float16)

In [177]:
a[0] = np()

1e-323

In [173]:
np.empty?

Docstring:
empty(shape, dtype=float, order='C')

Return a new array of given shape and type, without initializing entries.

Parameters
----------
shape : int or tuple of int
    Shape of the empty array, e.g., ``(2, 3)`` or ``2``.
dtype : data-type, optional
    Desired output data-type for the array, e.g, `numpy.int8`. Default is
    `numpy.float64`.
order : {'C', 'F'}, optional, default: 'C'
    Whether to store multi-dimensional data in row-major
    (C-style) or column-major (Fortran-style) order in
    memory.

Returns
-------
out : ndarray
    Array of uninitialized (arbitrary) data of the given shape, dtype, and
    order.  Object arrays will be initialized to None.

See Also
--------
empty_like : Return an empty array with shape and type of input.
ones : Return a new array setting values to one.
zeros : Return a new array setting values to zero.
full : Return a new array of given shape filled with value.


Notes
-----
`empty`, unlike `zeros`, does not set the array values to ze

In [ ]:
np.insert(np.array((5)))

In [167]:
np.insert?

Signature: np.insert(arr, obj, values, axis=None)
Docstring:
Insert values along the given axis before the given indices.

Parameters
----------
arr : array_like
    Input array.
obj : int, slice or sequence of ints
    Object that defines the index or indices before which `values` is
    inserted.

    .. versionadded:: 1.8.0

    Support for multiple insertions when `obj` is a single scalar or a
    sequence with one element (similar to calling insert multiple
    times).
values : array_like
    Values to insert into `arr`. If the type of `values` is different
    from that of `arr`, `values` is converted to the type of `arr`.
    `values` should be shaped so that ``arr[...,obj,...] = values``
    is legal.
axis : int, optional
    Axis along which to insert `values`.  If `axis` is None then `arr`
    is flattened first.

Returns
-------
out : ndarray
    A copy of `arr` with `values` inserted.  Note that `insert`
    does not occur in-place: a new array is returned. If
    `axis` is

In [158]:
for a,ano in enumerate(E.ano.unique()):
    _a = E[E.ano==ano]
    print(a,ano,sep=':',end=' ')
    for p,periodo in enumerate(_a.periodo.unique()):
        print(periodo,end='\n')
        _ap = _a[_a.periodo==periodo];
        u_i = _ap.id_turma.unique()
        
        for t,turma_id in enumerate(u_i):
            #print('\t',turma_id,end=':')
            _apt = _ap[_ap.id_turma == turma_id];
            _apt = _apt[~(_apt.descricao == 'INDEFERIDO')]
            _apt = _apt[~pd.isna(_apt.unidade)]
            for u,unidade in enumerate(_apt.unidade.unique()):
                #print(unidade,end=', ')
                _aptu = _apt[(_apt.unidade == unidade)]
                i = i_e[_aptu.id_componente_curricular.iloc[0]]
                z[i][a][p][u] = np.nanmean(_aptu.nota)
                #z[i][a][p][u] = _aptu.nota
                
                #c = c + 1
                #break
            #break
        #break
    #break
    print('',end='\n')
    

TypeError: 'float' object is not callable

In [157]:
# componente 0 
# ano 0
# periodo 0
# unidades 1,2,3

z[:][0][0][:]

array([[4.15319149, 5.38888889, 5.93333333],
       [6.08474576, 7.32857143, 6.98979592],
       [0.        , 0.        , 0.        ]])

In [ ]:
n_t

In [85]:
z = np.zeros((0,0,0,0,0), dtype=float)

In [ ]:
z

In [88]:
z[0][0][0][0][0] = 1

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
np.dtype(np.int6)

In [67]:
dt = np.dtype({'f15':(np.float,15)})

In [68]:
dt = np.dtype((np.float,15))

array([], shape=(0, 0, 0, 0, 0), dtype=float64)

In [73]:
dt.itemsize

120

In [65]:
dt['f15']

dtype('float64')

array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0.,

In [56]:
np.dtype?

Init signature: np.dtype(self, /, *args, **kwargs)
Docstring:     
dtype(obj, align=False, copy=False)

Create a data type object.

A numpy array is homogeneous, and contains elements described by a
dtype object. A dtype object can be constructed from different
combinations of fundamental numeric types.

Parameters
----------
obj
    Object to be converted to a data type object.
align : bool, optional
    Add padding to the fields to match what a C compiler would output
    for a similar C-struct. Can be ``True`` only if `obj` is a dictionary
    or a comma-separated string. If a struct dtype is being created,
    this also sets a sticky alignment flag ``isalignedstruct``.
copy : bool, optional
    Make a new copy of the data-type object. If ``False``, the result
    may just be a reference to a built-in data-type object.

See also
--------
result_type

Examples
--------
Using array-scalar type:

>>> np.dtype(np.int16)
dtype('int16')

Structured type, one field name 'f1', containing in

In [54]:
np.dtype()

numpy.dtype

In [ ]:
np.array()

<ipython-input-44-dfccc2875101>:1: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  np.dtype('Float64')


dtype('float64')

In [53]:
np.dtype((np.int,8)KKK)

dtype(('<i4', (8,)))

In [49]:
np.a

Signature: np.nanvar(a, axis=None, dtype=None, out=None, ddof=0, keepdims=<no value>)
Docstring:
Compute the variance along the specified axis, while ignoring NaNs.

Returns the variance of the array elements, a measure of the spread of
a distribution.  The variance is computed for the flattened array by
default, otherwise over the specified axis.

For all-NaN slices or slices with zero degrees of freedom, NaN is
returned and a `RuntimeWarning` is raised.

.. versionadded:: 1.8.0

Parameters
----------
a : array_like
    Array containing numbers whose variance is desired.  If `a` is not an
    array, a conversion is attempted.
axis : {int, tuple of int, None}, optional
    Axis or axes along which the variance is computed.  The default is to compute
    the variance of the flattened array.
dtype : data-type, optional
    Type to use in computing the variance.  For arrays of integer type
    the default is `float64`; for arrays of float types it is the same as
    the array type.
out : 

In [45]:
np.array(dtype='Float64',ndmin=4)

TypeError: array() missing required argument 'object' (pos 1)

In [36]:
import numpy as np 


array([nan, 4. , 1. , 4.5, 0. , 4. , 4. , 4. , 0. , 6.5, 3. , 5.5, 5.5,
       3. , 6.9, 8. , 0. , 5.5, 0. , 3.5, 5.8, 4. , 6. , 4.5, 2.5, 5.5,
       nan, 7.5, 3.5, nan, 0. , 3. , 6. , 5. , 5.5, 2.5, 7. , 3.5, nan,
       4. , 6.5, 3.5, 5. , 2. , 6. , 5.5, 3.5, 6. , 5.5, 3.5, 3.5])

,id_turma,id_componente_curricular,descricao,ano,periodo,unidade,nota,faltas_unidade,nome,id_curso,discente
0,57508932,27095,CANCELADO,2009,1,1,NaN,0,INTRODUCAO A TEORIA ECONOMICA,2000014,b44a6bfee4cbb74b5c6551ca0c0a4d06
4,57508932,27095,REPROVADO,2009,1,1,4.0,0,INTRODUCAO A TEORIA ECONOMICA,2000014,48acbaa9c243091a7d87418992ee9aeb
7,57508932,27095,REPROVADO,2009,1,1,1.0,0,INTRODUCAO A TEORIA ECONOMICA,2000014,dcdaeac3ed3d7b69817903d5909aea61
13,57508932,27095,APROVADO,2009,1,1,4.5,0,INTRODUCAO A TEORIA ECONOMICA,2000014,cd0bebea7a2b5951161b51f4f9c38635
16,57508932,27095,REPROVADO,2009,1,1,0.0,0,INTRODUCAO A TEORIA ECONOMICA,2000014,e28a76aae54521ab808fdab29435f7e5
19,57508932,27095,REPROVADO,2009,1,1,4.0,0,INTRODUCAO A TEORIA ECONOMICA,2000014,463f07e960cac94268b4b018c3779562
22,57508932,27095,TRANCADO,2009,1,1,4.0,0,INTRODUCAO A TEORIA ECONOMICA,2000014,11c408f36a1ad681d1697d7ed44938f1
25,57508932,27095,APROVADO,2009,1,1,4.0,0,INTRODUCAO A TEORIA ECONOMICA,2000014,398700163bd01833e313bcb61fafb539
28,57508932,27095,REPROVADO,2009,1,1,0.0,0,INTRODUCAO A TEORIA ECONOMICA,2000014,13c02416daba8404d6cb23dfc51db8d4
31,57508932,27095,EXCLUIDA,2009,1,1,6.5,0,INTRODUCAO A TEORIA ECONOMICA,2000014,f00e22186e037e7c21faaa356b5b2c22


0      NaN
4      4.0
7      1.0
13     4.5
16     0.0
19     4.0
22     4.0
25     4.0
28     0.0
31     6.5
34     3.0
37     5.5
40     5.5
43     3.0
48     6.9
51     8.0
54     0.0
57     5.5
60     0.0
63     3.5
66     5.8
69     4.0
72     6.0
75     4.5
79     2.5
82     5.5
86     NaN
90     7.5
94     3.5
99     NaN
102    0.0
105    3.0
108    6.0
111    5.0
115    5.5
118    2.5
121    7.0
124    3.5
127    NaN
130    4.0
134    6.5
137    3.5
140    5.0
143    2.0
146    6.0
149    5.5
152    3.5
155    6.0
158    5.5
161    3.5
164    3.5
Name: nota, dtype: float64

In [ ]:
_aptu

In [874]:
%matplotlib widget

plt.close(1)
f,a = plt.subplots(1,1,figsize=(7,3),num=1)

sns.histplot(_aptu.nota.to_numpy(),bins=np.arange(0,10.1,.1))
sns.set_theme(context='notebook',palette='deep',style='darkgrid')

a.set_ylabel('# discentes')
a.set_xlabel('Nota da unidade #%i' % unidade)
a.set_position(pos=[ .1, .2, .85, .6 ])
s = ''

In [877]:


s

'METODOLOGIA DA PESQUISA PARA O TRABALHO DE CONCLUSÃO DE CURSO EM CIÊNCIAS E TECNOLOGIA'

In [917]:
z = []
x = []
x.append('')
s = _aptu.nome.iloc[0]
S = s.split()

In [920]:

for s,ss in enumerate(S):   
    if len(x[-1]):
        x[-1] += x[-1] + ' ' + ss
    else:
        x[-1] += x[-1] + ' ' + ss
    
    print(s,ss)

0 METODOLOGIA
1 DA
2 PESQUISA
3 PARA
4 O
5 TRABALHO
6 DE
7 CONCLUSÃO
8 DE
9 CURSO
10 EM
11 CIÊNCIAS
12 E
13 TECNOLOGIA


In [895]:
z = []
x = []
x.append('')
s = _aptu.nome.iloc[0]
for s in s.split():
    x[-1] = x[-1] + ' ' + s
    if len(x[-1]) > 50:
        x.append('')
x
    

[' METODOLOGIA DA PESQUISA PARA O TRABALHO DE CONCLUSÃO',
 ' DE CURSO EM CIÊNCIAS E TECNOLOGIA']

In [874]:
a.set_title('%s' % s)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '')

In [871]:
#a.set_title('%s' % _aptu.nome.iloc[0])
#a.set_title(''.join([str(x) for x in range(10) for y in range(6)]))
#a.set_title('\n'.join(re.findall('\w*\s\w*\s\w*\s\w*\s',_aptu.nome.iloc[0])))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'METODOLOGIA DA PESQUISA PARA \nO TRABALHO DE CONCLUSÃO \nDE CURSO EM CIÊNCIAS ')

In [866]:
# Each line with 50 characteres
# or more up to 60 characteres not to split a string

s = _aptu.nome.iloc[0]

r = re.findall('(\w+\s?\w+\s?\w+\s?\w+\s?\w+\s?\w+\s?\w+)\s?',s)

print( len(s), s, sep='\n' )

print('')

for r in r:
    print(r)


86
METODOLOGIA DA PESQUISA PARA O TRABALHO DE CONCLUSÃO DE CURSO EM CIÊNCIAS E TECNOLOGIA

METODOLOGIA DA PESQUISA PARA O TRABALHO DE
CONCLUSÃO DE CURSO EM CIÊNCIAS E TECNOLOGIA


In [865]:
# Each line with 50 characteres
# or more up to 60 characteres not to split a string

s = _aptu.nome.iloc[0]

r = re.findall('(\w+\s?\w+\s?\w+\s?\w+\s?\w+\s?\w+\s?\w+)\s?',s)

print( len(s), s, sep='\n' )

print('')

for r in r:
    print(r)


86
METODOLOGIA DA PESQUISA PARA O TRABALHO DE CONCLUSÃO DE CURSO EM CIÊNCIAS E TECNOLOGIA

METODOLOGIA DA PESQUISA PARA O TRABALHO DE
CONCLUSÃO DE CURSO EM CIÊNCIAS E TECNOLOGIA


In [861]:
# Each line with 50 characteres
# or more up to 60 characteres not to split a string

s = _aptu.nome.iloc[0]

r = re.findall('(\w+\s?\w+\s?\w+\s?\w+\s?\w+\s?\w+\s?\w+)\s?',s)

print( len(s), s, sep='\n' )

for r in r:
    print(r)


86
METODOLOGIA DA PESQUISA PARA O TRABALHO DE CONCLUSÃO DE CURSO EM CIÊNCIAS E TECNOLOGIA
METODOLOGIA DA PESQUISA PARA O TRABALHO DE
CONCLUSÃO DE CURSO EM CIÊNCIAS E TECNOLOGIA


In [862]:
re.findall('\w*\s\w*\s\w*\s\w*\s',_aptu.nome.iloc[0])

['METODOLOGIA DA PESQUISA PARA ',
 'O TRABALHO DE CONCLUSÃO ',
 'DE CURSO EM CIÊNCIAS ']

Signature: re.split(pattern, string, maxsplit=0, flags=0)
Docstring:
Split the source string by the occurrences of the pattern,
returning a list containing the resulting substrings.  If
capturing parentheses are used in pattern, then the text of all
groups in the pattern are also returned as part of the resulting
list.  If maxsplit is nonzero, at most maxsplit splits occur,
and the remainder of the string is returned as the final element
of the list.
File:      c:\dev\64\anacoda\lib\re.py
Type:      function


In [758]:
re.split( '(\W)', _aptu.nome.iloc[0], flags=1 )

['METODOLOGIA',
 ' ',
 'DA',
 ' ',
 'PESQUISA',
 ' ',
 'PARA',
 ' ',
 'O',
 ' ',
 'TRABALHO',
 ' ',
 'DE',
 ' ',
 'CONCLUSÃO',
 ' ',
 'DE',
 ' ',
 'CURSO',
 ' ',
 'EM',
 ' ',
 'CIÊNCIAS',
 ' ',
 'E',
 ' ',
 'TECNOLOGIA']

In [661]:
_apt.faltas_unidade

9631    0
9632    0
9633    0
9636    0
9637    0
       ..
9972    0
9973    0
9977    0
9978    0
9979    0
Name: faltas_unidade, Length: 207, dtype: Int64

In [734]:
a.set_title?

Signature: a.set_title(label, fontdict=None, loc=None, pad=None, *, y=None, **kwargs)
Docstring:
Set a title for the axes.

Set one of the three available axes titles. The available titles
are positioned above the axes in the center, flush with the left
edge, and flush with the right edge.

Parameters
----------
label : str
    Text to use for the title

fontdict : dict
    A dictionary controlling the appearance of the title text,
    the default *fontdict* is::

       {'fontsize': rcParams['axes.titlesize'],
        'fontweight': rcParams['axes.titleweight'],
        'color': rcParams['axes.titlecolor'],
        'verticalalignment': 'baseline',
        'horizontalalignment': loc}

loc : {'center', 'left', 'right'}, default: :rc:`axes.titlelocation`
    Which title to set.

y : float, default: :rc:`axes.titley`
    Vertical axes loation for the title (1.0 is the top).  If
    None (the default), y is determined automatically to avoid
    decorators on the axes.

pad : float, default:

'METODOLOGIA DA PESQUISA PARA O TRABALHO DE CONCLUSÃO DE CURSO EM CIÊNCIAS E TECNOLOGIA'

,id_turma,id_componente_curricular,descricao,ano,periodo,unidade,nota,faltas_unidade,nome,id_curso,discente
0,57508932,27095,CANCELADO,2009,1,1,NaN,0,INTRODUCAO A TEORIA ECONOMICA,2000014,b44a6bfee4cbb74b5c6551ca0c0a4d06
4,57508932,27095,REPROVADO,2009,1,1,4.0,0,INTRODUCAO A TEORIA ECONOMICA,2000014,48acbaa9c243091a7d87418992ee9aeb
7,57508932,27095,REPROVADO,2009,1,1,1.0,0,INTRODUCAO A TEORIA ECONOMICA,2000014,dcdaeac3ed3d7b69817903d5909aea61
13,57508932,27095,APROVADO,2009,1,1,4.5,0,INTRODUCAO A TEORIA ECONOMICA,2000014,cd0bebea7a2b5951161b51f4f9c38635
16,57508932,27095,REPROVADO,2009,1,1,0.0,0,INTRODUCAO A TEORIA ECONOMICA,2000014,e28a76aae54521ab808fdab29435f7e5
19,57508932,27095,REPROVADO,2009,1,1,4.0,0,INTRODUCAO A TEORIA ECONOMICA,2000014,463f07e960cac94268b4b018c3779562
22,57508932,27095,TRANCADO,2009,1,1,4.0,0,INTRODUCAO A TEORIA ECONOMICA,2000014,11c408f36a1ad681d1697d7ed44938f1
25,57508932,27095,APROVADO,2009,1,1,4.0,0,INTRODUCAO A TEORIA ECONOMICA,2000014,398700163bd01833e313bcb61fafb539
28,57508932,27095,REPROVADO,2009,1,1,0.0,0,INTRODUCAO A TEORIA ECONOMICA,2000014,13c02416daba8404d6cb23dfc51db8d4
31,57508932,27095,EXCLUIDA,2009,1,1,6.5,0,INTRODUCAO A TEORIA ECONOMICA,2000014,f00e22186e037e7c21faaa356b5b2c22


<IntegerArray>
[1, 2, 3]
Length: 3, dtype: Int64

In [563]:
{'context':'paper','style':'ticks','palette':'colorblind'}

{'context': 'paper', 'style': 'ticks', 'palette': 'colorblind'}

In [562]:
sns.set_theme?

Signature:
sns.set_theme(
    context='notebook',
    style='darkgrid',
    palette='deep',
    font='sans-serif',
    font_scale=1,
    color_codes=True,
    rc=None,
)
Docstring:
Set multiple theme parameters in one step.

Each set of parameters can be set directly or temporarily, see the
referenced functions below for more information.

Parameters
----------
context : string or dict
    Plotting context parameters, see :func:`plotting_context`.
style : string or dict
    Axes style parameters, see :func:`axes_style`.
palette : string or sequence
    Color palette, see :func:`color_palette`.
font : string
    Font family, see matplotlib font manager.
font_scale : float, optional
    Separate scaling factor to independently scale the size of the
    font elements.
color_codes : bool
    If ``True`` and ``palette`` is a seaborn palette, remap the shorthand
    color codes (e.g. "b", "g", "r", etc.) to the colors from this palette.
rc : dict or None
    Dictionary of rc parameter mappin

In [560]:
%matplotlib widget

import seaborn as sns
from matplotlib import pyplot as plt
nota_bins = np.arange(0,10.2,.1)
plt.close(1)
(figure, ax) = plt.subplots(1,1,figsize=(9,3))
ss = sns.set_theme()
hhist = sns.histplot(_apt.nota.to_numpy(),bins=nota_bins,ax=ax)
ax.set_ylabel('Número de discentes')
ax.set_xlabel('Nota')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Nota')

In [491]:
a = sns.set_theme()

In [493]:
z = sns.set_style

In [504]:
z = sns.axes_style()

In [550]:
import re 
for x in np.unique([re.findall('(^[a-z]*)\.',z) for z in z.keys()]):
    print(x)

axes
figure
font
grid
image
lines
patch
text
xtick
ytick


In [505]:
z

{'axes.facecolor': '#EAEAF2',
 'axes.edgecolor': 'white',
 'axes.grid': True,
 'axes.axisbelow': True,
 'axes.labelcolor': '.15',
 'figure.facecolor': 'white',
 'grid.color': 'white',
 'grid.linestyle': '-',
 'text.color': '.15',
 'xtick.color': '.15',
 'ytick.color': '.15',
 'xtick.direction': 'out',
 'ytick.direction': 'out',
 'lines.solid_capstyle': 'round',
 'patch.edgecolor': 'w',
 'patch.force_edgecolor': True,
 'image.cmap': 'rocket',
 'font.family': ['sans-serif'],
 'font.sans-serif': ['Arial',
  'DejaVu Sans',
  'Liberation Sans',
  'Bitstream Vera Sans',
  'sans-serif'],
 'xtick.bottom': False,
 'xtick.top': False,
 'ytick.left': False,
 'ytick.right': False,
 'axes.spines.left': True,
 'axes.spines.bottom': True,
 'axes.spines.right': True,
 'axes.spines.top': True}

Signature: axes.set_aspect(aspect, adjustable=None, anchor=None, share=False)
Docstring:
Set the aspect of the axis scaling, i.e. the ratio of y-unit to x-unit.

Parameters
----------
aspect : {'auto'} or num
    Possible values:

    ========   =================================================
    value      description
    ========   =================================================
    'auto'     automatic; fill the position rectangle with data.
    num        a circle will be stretched such that the height
               is *num* times the width.  'equal' is a synonym
               for ``aspect=1``, i.e. same scaling for x and y.
    ========   =================================================

adjustable : None or {'box', 'datalim'}, optional
    If not ``None``, this defines which parameter will be adjusted to
    meet the required aspect. See `.set_adjustable` for further
    details.

anchor : None or str or 2-tuple of float, optional
    If not ``None``, this defines where th

In [447]:
axes.set_position()

Signature: axes.set_position(pos, which='both')
Docstring:
Set the axes position.

Axes have two position attributes. The 'original' position is the
position allocated for the Axes. The 'active' position is the
position the Axes is actually drawn at. These positions are usually
the same unless a fixed aspect is set to the Axes. See `.set_aspect`
for details.

Parameters
----------
pos : [left, bottom, width, height] or `~matplotlib.transforms.Bbox`
    The new position of the in `.Figure` coordinates.

which : {'both', 'active', 'original'}, default: 'both'
    Determines which position variables to change.
File:      c:\dev\64\anacoda\lib\site-packages\matplotlib\axes\_base.py
Type:      method


In [442]:
axes.set_ylabel('ok')

Text(91.97222222222221, 0.5, 'ok')

In [439]:
set_label('ok')
plt.draw()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [392]:
np.nanmean(_apt.nota.to_numpy())

,id_turma,id_componente_curricular,descricao,ano,periodo,unidade,nota,faltas_unidade,nome,id_curso,discente
9631,57663033,24772,APROVADO,2020,5,1,9.8,0,LEGISLACAO E SEGURANCA DO TRABALHO,85322571,b86c263574ebe84d4fb65a5d53c660f2
9632,57663033,24772,APROVADO,2020,5,2,9.8,0,LEGISLACAO E SEGURANCA DO TRABALHO,85322571,b86c263574ebe84d4fb65a5d53c660f2
9633,57663033,24772,APROVADO,2020,5,3,9.0,0,LEGISLACAO E SEGURANCA DO TRABALHO,85322571,b86c263574ebe84d4fb65a5d53c660f2
9634,57663033,24772,INDEFERIDO,2020,5,<NA>,NaN,<NA>,LEGISLACAO E SEGURANCA DO TRABALHO,10320810,5ab5d06a55689401dd5c8e3d98a70492
9635,57663033,24772,INDEFERIDO,2020,5,<NA>,NaN,<NA>,LEGISLACAO E SEGURANCA DO TRABALHO,2000030,b3ff8a8318cfd90347e8f34fb3cbe8ac
...,...,...,...,...,...,...,...,...,...,...,...
9975,57663033,24772,INDEFERIDO,2020,5,<NA>,NaN,<NA>,LEGISLACAO E SEGURANCA DO TRABALHO,2000030,2241314b95b422cc40cb2370188310a8
9976,57663033,24772,INDEFERIDO,2020,5,<NA>,NaN,<NA>,LEGISLACAO E SEGURANCA DO TRABALHO,10320810,25c8907b429c654ed810ca9031c14549
9977,57663033,24772,APROVADO,2020,5,1,9.0,0,LEGISLACAO E SEGURANCA DO TRABALHO,2000029,cffd52320c1c3359b60b835ffd65bbb1
9978,57663033,24772,APROVADO,2020,5,2,9.8,0,LEGISLACAO E SEGURANCA DO TRABALHO,2000029,cffd52320c1c3359b60b835ffd65bbb1


8.370531400966183

In [384]:
_apt.periodo.unique()

array([5], dtype=int64)

In [350]:
eapt

,id_turma,id_componente_curricular,descricao,ano,periodo,unidade,nota,faltas_unidade,nome,id_curso,discente
224362,57662510,51589,EXCLUIDA,2020,1,1,NaN,0,BIOSSEGURANÇA E BIOÉTICA,85341136,b474bb3ec93a150a63e3d3006c36860d
224363,57662510,51589,EXCLUIDA,2020,1,2,NaN,0,BIOSSEGURANÇA E BIOÉTICA,85341136,b474bb3ec93a150a63e3d3006c36860d
224364,57662510,51589,EXCLUIDA,2020,1,3,NaN,0,BIOSSEGURANÇA E BIOÉTICA,85341136,b474bb3ec93a150a63e3d3006c36860d


In [340]:
ea.ano

2807      2020
2808      2020
2809      2020
2810      2020
2811      2020
          ... 
658191    2020
658192    2020
658193    2020
658194    2020
658195    2020
Name: ano, Length: 11293, dtype: int64

In [296]:
import numpy as np
np.nanmean(E.nota)

4.992010876498271

In [88]:
pd.DataFrame.groupby?

Signature:
pd.DataFrame.groupby(
    self,
    by=None,
    axis=0,
    level=None,
    as_index: bool = True,
    sort: bool = True,
    group_keys: bool = True,
    squeeze: bool = False,
    observed: bool = False,
) -> 'groupby_generic.DataFrameGroupBy'
Docstring:
Group DataFrame using a mapper or by a Series of columns.

A groupby operation involves some combination of splitting the
object, applying a function, and combining the results. This can be
used to group large amounts of data and compute operations on these
groups.

Parameters
----------
by : mapping, function, label, or list of labels
    Used to determine the groups for the groupby.
    If ``by`` is a function, it's called on each value of the object's
    index. If a dict or Series is passed, the Series or dict VALUES
    will be used to determine the groups (the Series' values are first
    aligned; see ``.align()`` method). If an ndarray is passed, the
    values are used as-is determine the groups. A label or list o